In [20]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [22]:
df = pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])

In [23]:
df.dropna(inplace=True)

In [24]:
df.shape

(714, 3)

In [25]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [26]:
x = df.iloc[:,1:]
y = df.iloc[:,0]

In [27]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.2)

In [28]:
x_train.head()

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542
253,30.0,16.1000
719,33.0,7.7750
666,25.0,13.0000


In [29]:
clf = DecisionTreeClassifier()

In [30]:
clf.fit(x_train,y_train)
ypred = clf.predict(x_test)

In [31]:
accuracy_score(y_test,ypred)

0.6433566433566433

In [32]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

np.float64(0.6373043818466353)

In [33]:
kbin_age = KBinsDiscretizer(n_bins=10,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=10,encode='ordinal',strategy='quantile')

In [35]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [36]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf  = trf.fit_transform(x_test)

c:\Users\Gagan\Desktop\Python\Jupy\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
c:\Users\Gagan\Desktop\Python\Jupy\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
c:\Users\Gagan\Desktop\Python\Jupy\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current defaul

In [37]:
trf.named_transformers_['first'].n_bins_

array([10])

In [38]:
trf.named_transformers_['first'].bin_edges_

array([array([ 1. , 11. , 17. , 20.6, 24. , 28. , 30.1, 34.4, 38.6, 47.8, 62. ])],
      dtype=object)

In [39]:
output = pd.DataFrame({
    'age':x_train['Age'],
    'age_trf':x_train['Fare'],
    'fare':x_train['Fare'],
    'fare_trf':x_train_trf[:,1]
})

In [42]:
output['age_labels']= pd.cut(x=x_train['Age'],bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels']= pd.cut(x=x_train['Fare'],bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [43]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
747,30.0,13.0000,13.0000,4.0,"(28.0, 30.1]","(10.5, 14.454]"
626,57.0,12.3500,12.3500,3.0,"(47.8, 62.0]","(10.5, 14.454]"
609,40.0,153.4625,153.4625,9.0,"(38.6, 47.8]","(57.783, 512.329]"
467,56.0,26.5500,26.5500,6.0,"(47.8, 62.0]","(22.62, 28.39]"
162,26.0,7.7750,7.7750,1.0,"(24.0, 28.0]","(7.743, 7.925]"
